# Phase 9: High-Frequency & Algorithmic Execution

The algorithmic execution layer for speed-sensitive strategies.

This notebook covers:
- **Order book mechanics** — bids, asks, depth
- **TWAP and VWAP** — algorithmic order types
- **Market impact modeling** — understanding slippage
- **Arbitrage detection** — cross-exchange opportunities

---

```bash
pip install pandas numpy matplotlib
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

---
## 9.1 Order Book Mechanics

Understanding the order book is fundamental to algorithmic trading.

In [ ]:
@dataclass
class OrderBookLevel:
    price: float
    quantity: float

class OrderBook:
    def __init__(self):
        self.bids: List[OrderBookLevel] = []  # sorted descending by price
        self.asks: List[OrderBookLevel] = []  # sorted ascending by price
    
    @property
    def best_bid(self) -> float:
        return self.bids[0].price if self.bids else 0
    
    @property
    def best_ask(self) -> float:
        return self.asks[0].price if self.asks else float('inf')
    
    @property
    def mid_price(self) -> float:
        return (self.best_bid + self.best_ask) / 2
    
    @property
    def spread(self) -> float:
        return self.best_ask - self.best_bid
    
    @property
    def spread_bps(self) -> float:
        return self.spread / self.mid_price * 10000
    
    def simulate_market_buy(self, quantity: float) -> Tuple[float, float]:
        """Simulate a market buy order. Returns (avg_price, total_cost)."""
        remaining = quantity
        total_cost = 0
        for level in self.asks:
            fill_qty = min(remaining, level.quantity)
            total_cost += fill_qty * level.price
            remaining -= fill_qty
            if remaining <= 0:
                break
        filled = quantity - remaining
        avg_price = total_cost / filled if filled > 0 else 0
        return avg_price, total_cost
    
    def simulate_market_sell(self, quantity: float) -> Tuple[float, float]:
        """Simulate a market sell order. Returns (avg_price, total_proceeds)."""
        remaining = quantity
        total_proceeds = 0
        for level in self.bids:
            fill_qty = min(remaining, level.quantity)
            total_proceeds += fill_qty * level.price
            remaining -= fill_qty
            if remaining <= 0:
                break
        filled = quantity - remaining
        avg_price = total_proceeds / filled if filled > 0 else 0
        return avg_price, total_proceeds


def generate_order_book(mid_price=100, spread_bps=10, depth_levels=10):
    """Generate a realistic order book."""
    ob = OrderBook()
    spread = mid_price * spread_bps / 10000
    
    # Generate asks (ascending from best ask)
    ask_start = mid_price + spread / 2
    for i in range(depth_levels):
        price = ask_start + i * 0.05
        # Quantity increases with distance from mid (more liquidity away from touch)
        quantity = np.random.exponential(100) * (1 + i * 0.5)
        ob.asks.append(OrderBookLevel(price, quantity))
    
    # Generate bids (descending from best bid)
    bid_start = mid_price - spread / 2
    for i in range(depth_levels):
        price = bid_start - i * 0.05
        quantity = np.random.exponential(100) * (1 + i * 0.5)
        ob.bids.append(OrderBookLevel(price, quantity))
    
    return ob


ob = generate_order_book(mid_price=100, spread_bps=10)

print(f"Best Bid: ${ob.best_bid:.2f}")
print(f"Best Ask: ${ob.best_ask:.2f}")
print(f"Mid Price: ${ob.mid_price:.2f}")
print(f"Spread: ${ob.spread:.4f} ({ob.spread_bps:.1f} bps)")

# Visualize order book
fig, ax = plt.subplots(figsize=(12, 6))

bid_prices = [l.price for l in ob.bids]
bid_qty = [l.quantity for l in ob.bids]
ask_prices = [l.price for l in ob.asks]
ask_qty = [l.quantity for l in ob.asks]

ax.barh(bid_prices, bid_qty, height=0.04, color='green', alpha=0.6, label='Bids')
ax.barh(ask_prices, [-q for q in ask_qty], height=0.04, color='red', alpha=0.6, label='Asks')
ax.axhline(y=ob.mid_price, color='gray', linestyle='--', label=f'Mid ${ob.mid_price:.2f}')
ax.set_xlabel('Quantity')
ax.set_ylabel('Price ($)')
ax.set_title('Order Book Depth', fontsize=14)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Market impact simulation
order_sizes = [50, 100, 200, 500, 1000, 2000, 5000]
slippages = []

print(f"{'Order Size':>12} {'Avg Price':>12} {'Slippage':>12}")
print('-' * 40)

for size in order_sizes:
    avg_price, _ = ob.simulate_market_buy(size)
    slippage_bps = (avg_price - ob.best_ask) / ob.mid_price * 10000
    slippages.append(slippage_bps)
    print(f"{size:>12} ${avg_price:>11.4f} {slippage_bps:>11.2f} bps")

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(order_sizes, slippages, color='red', marker='o', linewidth=2)
ax.set_xlabel('Order Size (shares)')
ax.set_ylabel('Slippage (bps)')
ax.set_title('Market Impact: Slippage vs Order Size', fontsize=13)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 9.2 TWAP and VWAP Algorithms

Execute large orders by slicing them into smaller pieces over time.

In [ ]:
def generate_intraday_data(n_minutes=390):  # 6.5 hours
    """Generate intraday price and volume data."""
    # U-shaped volume curve (high at open/close)
    minutes = np.arange(n_minutes)
    volume_curve = 1 + 2 * np.exp(-((minutes - 0) / 30)**2) + \
                   2 * np.exp(-((minutes - n_minutes) / 30)**2)
    volume = np.random.poisson(volume_curve * 1000)
    
    # Price with slight trend and noise
    returns = np.random.normal(0.00001, 0.0005, n_minutes)
    prices = 100 * np.exp(np.cumsum(returns))
    
    return pd.DataFrame({
        'price': prices,
        'volume': volume
    }, index=pd.date_range('2024-01-01 09:30', periods=n_minutes, freq='min'))


def execute_twap(total_qty: float, data: pd.DataFrame, n_slices: int = 20):
    """Time-Weighted Average Price: split order evenly over time."""
    slice_qty = total_qty / n_slices
    slice_interval = len(data) // n_slices
    
    fills = []
    for i in range(n_slices):
        idx = i * slice_interval
        price = data['price'].iloc[idx]
        fills.append({'time': data.index[idx], 'price': price, 'qty': slice_qty})
    
    fills_df = pd.DataFrame(fills)
    avg_price = (fills_df['price'] * fills_df['qty']).sum() / fills_df['qty'].sum()
    return fills_df, avg_price


def execute_vwap(total_qty: float, data: pd.DataFrame, n_slices: int = 20):
    """Volume-Weighted Average Price: split order proportional to volume."""
    slice_interval = len(data) // n_slices
    
    # Calculate volume in each slice
    slice_volumes = []
    for i in range(n_slices):
        start_idx = i * slice_interval
        end_idx = (i + 1) * slice_interval
        vol = data['volume'].iloc[start_idx:end_idx].sum()
        slice_volumes.append(vol)
    
    total_vol = sum(slice_volumes)
    
    fills = []
    for i in range(n_slices):
        idx = i * slice_interval
        price = data['price'].iloc[idx]
        qty = total_qty * (slice_volumes[i] / total_vol)
        fills.append({'time': data.index[idx], 'price': price, 'qty': qty})
    
    fills_df = pd.DataFrame(fills)
    avg_price = (fills_df['price'] * fills_df['qty']).sum() / fills_df['qty'].sum()
    return fills_df, avg_price


data = generate_intraday_data()
total_qty = 10000

twap_fills, twap_price = execute_twap(total_qty, data)
vwap_fills, vwap_price = execute_vwap(total_qty, data)

# Calculate market VWAP
market_vwap = (data['price'] * data['volume']).sum() / data['volume'].sum()

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price
axes[0].plot(data.index, data['price'], color='steelblue', linewidth=1)
axes[0].axhline(y=twap_price, color='orange', linestyle='--', label=f'TWAP: ${twap_price:.4f}')
axes[0].axhline(y=vwap_price, color='green', linestyle='--', label=f'VWAP: ${vwap_price:.4f}')
axes[0].axhline(y=market_vwap, color='gray', linestyle=':', label=f'Market VWAP: ${market_vwap:.4f}')
axes[0].set_ylabel('Price ($)')
axes[0].set_title('TWAP vs VWAP Execution', fontsize=14)
axes[0].legend()

# Volume profile
axes[1].bar(data.index, data['volume'], width=0.0005, color='gray', alpha=0.5)
axes[1].set_ylabel('Market Volume')

# Fill comparison
axes[2].bar(twap_fills['time'], twap_fills['qty'], width=0.005, alpha=0.5, color='orange', label='TWAP fills')
axes[2].bar(vwap_fills['time'], vwap_fills['qty'], width=0.005, alpha=0.5, color='green', label='VWAP fills')
axes[2].set_ylabel('Order Qty')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"Execution comparison for {total_qty:,} shares:")
print(f"  TWAP avg price:   ${twap_price:.4f}")
print(f"  VWAP avg price:   ${vwap_price:.4f}")
print(f"  Market VWAP:      ${market_vwap:.4f}")
print(f"  VWAP vs market:   {(vwap_price/market_vwap - 1)*10000:.2f} bps")

---
## 9.3 Arbitrage Detection

Find price discrepancies across exchanges.

In [ ]:
def generate_multi_exchange_prices(n=1000, base_price=100, n_exchanges=3):
    """Generate correlated prices across multiple exchanges."""
    # Base price process
    returns = np.random.normal(0, 0.001, n)
    base = base_price * np.exp(np.cumsum(returns))
    
    prices = {}
    for i in range(n_exchanges):
        # Each exchange has slight deviation + noise
        deviation = np.random.normal(0, 0.0002, n)  # persistent deviation
        noise = np.random.normal(0, 0.0001, n)      # random noise
        prices[f'Exchange_{i+1}'] = base * (1 + np.cumsum(deviation) * 0.001 + noise)
    
    return pd.DataFrame(prices)


def detect_arbitrage(prices_df, threshold_bps=5):
    """Detect arbitrage opportunities when price difference exceeds threshold."""
    opportunities = []
    exchanges = prices_df.columns
    
    for i in range(len(prices_df)):
        row = prices_df.iloc[i]
        min_ex = row.idxmin()
        max_ex = row.idxmax()
        min_price = row[min_ex]
        max_price = row[max_ex]
        
        spread_bps = (max_price - min_price) / min_price * 10000
        
        if spread_bps > threshold_bps:
            opportunities.append({
                'idx': i,
                'buy_exchange': min_ex,
                'sell_exchange': max_ex,
                'buy_price': min_price,
                'sell_price': max_price,
                'spread_bps': spread_bps
            })
    
    return pd.DataFrame(opportunities)


prices = generate_multi_exchange_prices()
arb_opps = detect_arbitrage(prices, threshold_bps=3)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Prices
for col in prices.columns:
    axes[0].plot(prices[col], linewidth=0.8, alpha=0.8, label=col)
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Cross-Exchange Price Comparison', fontsize=14)
axes[0].legend()

# Spread (max - min)
spread = (prices.max(axis=1) - prices.min(axis=1)) / prices.min(axis=1) * 10000
axes[1].plot(spread, color='purple', linewidth=0.8)
axes[1].axhline(y=3, color='red', linestyle='--', label='Threshold (3 bps)')
if len(arb_opps) > 0:
    axes[1].scatter(arb_opps['idx'], arb_opps['spread_bps'], color='red', s=20, zorder=5)
axes[1].set_ylabel('Cross-Exchange Spread (bps)')
axes[1].set_xlabel('Time')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Arbitrage opportunities detected: {len(arb_opps)}")
if len(arb_opps) > 0:
    print(f"Average spread: {arb_opps['spread_bps'].mean():.2f} bps")
    print(f"Max spread: {arb_opps['spread_bps'].max():.2f} bps")
    print(f"\nSample opportunities:")
    print(arb_opps.head())

### Exercise 9.3

1. Add transaction costs (2 bps per trade) to the arbitrage calculation. How many opportunities remain profitable?
2. Implement a latency simulation: your detection is 10ms delayed. How many opportunities would you miss?
3. Calculate the total P&L if you traded every arbitrage opportunity with 1000 shares.

In [ ]:
# YOUR CODE HERE


---
## 9.4 Comprehension Check

1. The spread is 10 bps. You need to cross the spread to execute immediately. What's your instant cost for a round trip (buy then sell)?
2. Why does VWAP typically outperform TWAP? When might TWAP be preferred?
3. A 5000-share order has 15 bps of market impact. If you split it into 10 orders of 500, will total impact be more or less than 15 bps? Why?
4. Cross-exchange arbitrage requires speed. What infrastructure is needed to capture these opportunities?
5. Why do HFT firms co-locate their servers at exchange data centers?

In [ ]:
# YOUR ANSWERS HERE
